In [ ]:
import subprocess
from time import sleep

import numpy as np
import pynvml
from tqdm import tqdm
import time
import os

pynvml.nvmlInit()
last_task_t = [0] * 10


def get_free_MB(i):
    handle = pynvml.nvmlDeviceGetHandleByIndex(i)
    return pynvml.nvmlDeviceGetMemoryInfo(handle).free / 1024 / 1024


def get_ps_num(i):
    handle = pynvml.nvmlDeviceGetHandleByIndex(i)
    gpu_ps = pynvml.nvmlDeviceGetComputeRunningProcesses(handle)
    return len(gpu_ps)


def get_avail_gpu(size=10000):
    num = pynvml.nvmlDeviceGetCount()
    free = []
    for _ in range(60):
        sleep(1)
        free.append([get_free_MB(i) for i in range(num)])
    free = np.array(free).T.tolist()
    avail_gpus = []
    for i in range(num):
        if i == 3:
            continue
        if min(free[i]) > size and time.time() - last_task_t[i] > 600:
            avail_gpus.append(i)
    return avail_gpus

In [7]:
tasks = []
# for tol in [30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95]:
#     tasks.append((f"wfgan_tol{tol}", "wfgan"))
# for gan in [1, 3, 5, 7, 9]:
#     for tol in [30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95]:
#         tasks.append((f"switch_gan{gan:02d}_tol{tol}_front{10-gan:02d}", "switch_gan_front"))
# for p in ["equal", "random"]:
#     for tamaraw in range(2):
#         for regulartor in range(2):
#             for front in range(2):
#                 for wfgan in range(2):
#                     if tamaraw + regulartor + front + wfgan > 0:
#                         name = f"switch_{p}"
#                         if front:
#                             name += "_front"
#                         if wfgan:
#                             name += "_wfgan"
#                         if tamaraw:
#                             name += "_tamaraw"
#                         if regulartor:
#                             name += "_regulartor"
#                         tasks.append((name, f"switch_{p}"))


# tasks.extend([(name, "base") for name in ["undefend", "front", "tamaraw", "wfgan", "regulartor"]])

for tol in [1,2,3,4,5,6,7,8,9]:
    tasks.append(
        f"gan300_tol{tol}"
    )
tasks = [f"python train.py -d sdfow --train {task} --note bach" for task in tasks]
idx = 0
while idx < len(tasks):
    for gpu in [0, 1, 2, 4, 5, 6, 7]:
        if idx >= len(tasks):
            break
        print(f"{tasks[idx]}  -g {gpu} &")
        idx += 1
    print("wait")

python train.py -d sdfow --train gan300_tol1 --note bach  -g 0 &
python train.py -d sdfow --train gan300_tol2 --note bach  -g 1 &
python train.py -d sdfow --train gan300_tol3 --note bach  -g 2 &
python train.py -d sdfow --train gan300_tol4 --note bach  -g 4 &
python train.py -d sdfow --train gan300_tol5 --note bach  -g 5 &
python train.py -d sdfow --train gan300_tol6 --note bach  -g 6 &
python train.py -d sdfow --train gan300_tol7 --note bach  -g 7 &
wait
python train.py -d sdfow --train gan300_tol8 --note bach  -g 0 &
python train.py -d sdfow --train gan300_tol9 --note bach  -g 1 &
wait


In [ ]:
avail_gpus = []
for task, note in tqdm(tasks):
    log_dir = f"run/20240308{note}/RF/train_undefend_d{task}"
    if os.path.exists(log_dir):
        print(f"skip {task}")
        continue
    while not avail_gpus:
        avail_gpus = get_avail_gpu()

    gpu = avail_gpus.pop()
    subprocess.Popen(
        f"python train.py --train {task} --note {note} -g {gpu}",
        shell=True,
        stdout=subprocess.DEVNULL,
    )
    last_task_t[gpu] = time.time()

In [2]:
tasks = []
for train in [
    "undefend&front&tamaraw&wfgan&regulartor",
    "undefend",
    "front",
    "tamaraw",
    "regulartor",
    "wfgan",
    "ezpadding",
    "ezfixed",
    "ezfixedrate",
    "ezlinear",
    "adptamaraw",
]:
    for test in [
        "undefend",
        "front",
        "tamaraw",
        "regulartor",
        "wfgan",
        "ezpadding",
        "ezfixed",
        "ezfixedrate",
        "ezlinear",
        "adptamaraw",
    ]:
        tasks.append(
            f'python evaluate.py --attack DF --note 20240316ourbase --train "{train}" --test "{test}"'
        )
# idx = 0
# while idx < len(tasks):
#     for gpu in [0, 1, 2, 4, 5]:
#         if idx >= len(tasks):
#             break
#         print(f"{tasks[idx]}  -g {gpu} &")
#         idx += 1
#     print("wait")
idx = 0
while idx < len(tasks):
    for gpu in [0, 1, 2, 4, 5, 6, 7, 8, 9]:
        if idx >= len(tasks):
            break
        print(f"{tasks[idx]}  -g {gpu} &")
        idx += 1
    print("wait")

python evaluate.py --attack DF --note 20240316ourbase --train "undefend&front&tamaraw&wfgan&regulartor" --test "undefend"  -g 0 &
python evaluate.py --attack DF --note 20240316ourbase --train "undefend&front&tamaraw&wfgan&regulartor" --test "front"  -g 1 &
python evaluate.py --attack DF --note 20240316ourbase --train "undefend&front&tamaraw&wfgan&regulartor" --test "tamaraw"  -g 2 &
python evaluate.py --attack DF --note 20240316ourbase --train "undefend&front&tamaraw&wfgan&regulartor" --test "regulartor"  -g 4 &
python evaluate.py --attack DF --note 20240316ourbase --train "undefend&front&tamaraw&wfgan&regulartor" --test "wfgan"  -g 5 &
python evaluate.py --attack DF --note 20240316ourbase --train "undefend&front&tamaraw&wfgan&regulartor" --test "ezpadding"  -g 6 &
python evaluate.py --attack DF --note 20240316ourbase --train "undefend&front&tamaraw&wfgan&regulartor" --test "ezfixed"  -g 7 &
python evaluate.py --attack DF --note 20240316ourbase --train "undefend&front&tamaraw&wfgan&reg

In [1]:
import pandas as pd

overhead = pd.read_csv("log/overhead_private.csv", index_col=0, header=None).T.round(2)
overhead.index = ["time", "data"]
trains = [
    "undefend",
    "front",
    "tamaraw",
    "regulartor",
    "wfgan",
    "undefend&front&tamaraw&regulartor&wfgan",
]
tests = [
    "undefend",
    "front",
    "tamaraw",
    "regulartor",
    "wfgan",
    "ezpadding",
    "ezfixed",
    "ezlinear",
    "adptamaraw",
    "ezfixedrate",
]
result = pd.DataFrame(columns=tests, index=trains)
for train in trains:
    for test in tests:
        logfile = f"/home/zhangyixiang/wfp/run/test/20240313multi/undefend/{train}/{test}.log"
        with open(logfile, "r") as f:
            log = f.read().strip()
        result.loc[train, test] = float(eval(log)["accuracy"])
# result.index[5] = "5 - Mixed"
result.index = [
    "Undefend",
    "Front",
    "Tamaraw",
    "Regulartor",
    "WFGAN",
    "Mixed-5",
]
# cat result and overhead
result = pd.concat([result, overhead])

FileNotFoundError: [Errno 2] No such file or directory: 'log/overhead_private.csv'

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

result = result.astype(float)
plt.figure(figsize=(6, 4))

ax = sns.heatmap(result.round(2), cmap="YlGnBu", annot=True)

# ax.set_title("Accuracy over different tol parameter")
ax.set_xlabel("Test")
ax.xaxis.set_ticks_position("top")
ax.xaxis.set_label_position("top")
ax.xaxis.set_tick_params(rotation=45)
ax.set_ylabel("Train")
ax.yaxis.set_tick_params(rotation=0)
plt.tight_layout()
# plt.show()
plt.savefig(f"fig/private.pdf")